# Explore here

In [31]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from dotenv import load_dotenv

# load the .env file variables
load_dotenv(dotenv_path='../.env')



True

In [36]:
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')

print(client_secret)

your_client_secret


In [34]:
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
spotify = spotipy.Spotify(client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

In [27]:
#search for led zeppelin

ledzep = 'spotify:artist:36QJpDe2go2KgaRleHCDTp'

results = spotify.artist_top_tracks(ledzep)

for i in results['tracks'][:10]:
    print(i['name'])

SpotifyOauthError: error: invalid_client, error_description: Invalid client